In [1]:
import os
import pandas as pd
import shutil
import re

In [ ]:
dir_with_ner_files = "/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_swapped"

In [ ]:
# source_dir = dir_with_ner_files
# destination_dir = "/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_swapped_sentences_extracted"

# shutil.copytree(source_dir, destination_dir)

In [ ]:
parallel_dataset_bruk_dataset = pd.read_csv("/Users/linndfors/study/diploma/ner_for_fem/data/csv_files_with_par_sentences/bruk_parallel.csv")
parallel_dataset_ng_dataset = pd.read_csv("/Users/linndfors/study/diploma/ner_for_fem/data/csv_files_with_par_sentences/ng_parallel.csv")

In [ ]:
parallel_dataset_bruk_dataset.head()

In [ ]:
def read_csv(csv_path, file_name):
    """Read CSV file and return a list of sentence IDs that need to be kept."""
    df = pd.read_csv(csv_path)
    df = df[df['original_file_name'] == file_name.replace("_1.txt", ".txt")]
    sentence_positions = {row['changed_sent_id']: row['changed_sentence'] for _, row in df.iterrows()}
    return sentence_positions


def filter_txt(txt_path, sentence_positions):
    """Filter sentences in the TXT file based on the CSV list and return updated text."""
    with open(txt_path, "r", encoding="utf-8") as f:
        sentences = f.readlines()

    sentence_map = {}
    
    start_ind = 0
    end_index = 0
    for i, sentence in enumerate(sentences):
        end_index += len(sentence)
        if i in sentence_positions.keys():
            selected_sent = sentence_positions[i]

            sentence_map[i] = (selected_sent, start_ind, end_index)
        start_ind += len(sentence)
    return sentence_map

def update_ann(ann_path, sentence_map):
    """Update annotation indices based on the new text mapping."""
    with open(ann_path, "r", encoding="utf-8") as f:
        ann_lines = f.readlines()
    
    updated_ann = []

    add_ind_to_this_sent = {}
    start_ind_len = 0

    for line in ann_lines:
        
        match = re.match(r"(T\d+)\t(\w+)\t(\d+)\s(\d+)\t(.+)", line)
        if match:
            tid, label, start, end, text = match.groups()
            start, end = int(start), int(end)

            
            for sent_ind, indxs in sentence_map.items():
                selected_sent, sentence_start_ind, sentence_end_index = indxs
                if start >= sentence_start_ind and end <= sentence_end_index:

                    if sent_ind not in add_ind_to_this_sent.keys():
                        add_ind_to_this_sent[sent_ind] = start_ind_len
                        start_ind_len += len(selected_sent)

                    updated_start = start - sentence_start_ind + add_ind_to_this_sent[sent_ind]
                    updated_end = start - sentence_start_ind + len(text) + add_ind_to_this_sent[sent_ind]
                    
                    updated_ann.append(f"{tid}\t{label}\t{updated_start} {updated_end}\t{text}\n")

    return updated_ann

def create_new_txt_and_ann(txt_path, ann_path, sentence_positions):
    """Create new TXT and ANN files with only the selected sentences and updated annotations."""
    sentence_map = filter_txt(txt_path, sentence_positions)
    updated_ann = update_ann(ann_path, sentence_map)
    
    new_txt_path = txt_path.replace(".txt", "_filtered.txt")
    new_txt = "\n".join(list(sentence_positions.values()))
    if new_txt:
        with open(new_txt_path, "w", encoding="utf-8") as f:
            f.write(new_txt)
        
        if updated_ann:
            new_ann_path = ann_path.replace(".ann", "_filtered.ann")
            with open(new_ann_path, "w", encoding="utf-8") as f:
                f.writelines(updated_ann)
            
    # print(f"New filtered TXT saved at: {new_txt_path}")
    # print(f"New filtered ANN saved at: {new_ann_path}")

In [ ]:
# # Example usage:

# # /Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_swapped_sentences_extracted/bruk_changed/c0c436cc9cb6_1_filtered.txt
# # /Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_swapped_sentences_extracted/bruk_changed/232be2c6b5c2_1.txt
# txt_path = "/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_swapped_sentences_extracted/bruk_changed/232be2c6b5c2_1.txt"
# ann_path = "/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_swapped_sentences_extracted/bruk_changed/232be2c6b5c2_1.ann"
# csv_path = "/Users/linndfors/study/diploma/ner_for_fem/data/csv_files_with_par_sentences/bruk_parallel.csv"
# file_name = Path(txt_path).name

# # Read the sentence positions from the CSV file
# sentence_positions = read_csv(csv_path, file_name)

# # Create the new TXT and ANN files with selected sentences and updated annotations
# create_new_txt_and_ann(txt_path, ann_path, sentence_positions)

In [ ]:
import tqdm

csv_path = "/Users/linndfors/study/diploma/ner_for_fem/data/csv_files_with_par_sentences/ng_parallel.csv"
directory_path = '/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_swapped_sentences_extracted/ng_changed'

files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) 
                  if os.path.isfile(os.path.join(directory_path, f)) and ".txt" in f]

In [ ]:
fd7413336034_1_filtered

In [ ]:
# for txt_path in tqdm.tqdm(files):
    
#     ann_path = txt_path.replace(".txt", ".ann")
#     file_name = Path(txt_path).name

#     sentence_positions = read_csv(csv_path, file_name)
#     create_new_txt_and_ann(txt_path, ann_path, sentence_positions)

In [ ]:
import re

def check_entity_indices(txt_path, ann_path):
    bad_line = 0
    """Check whether entity indices in the .ann file match the text."""
    with open(txt_path, "r", encoding="utf-8") as f:
        text = f.read().strip()  # Read the full text and strip extra spaces or newlines

    with open(ann_path, "r", encoding="utf-8") as f:
        ann_lines = f.readlines()

    # Process each annotation line
    for line in ann_lines:
        match = re.match(r"(T\d+)\t(\w+)\t(\d+)\s(\d+)\t(.+)", line)
        if match:
            tid, label, start, end, entity_text = match.groups()
            start, end = int(start), int(end)

            # Extract the corresponding substring from the text
            substring = text[start:end]

            # Compare extracted substring with the annotation's text
            if substring == entity_text:
                pass
                # print(f"Annotation {tid} is correct: '{entity_text}' found at indices {start}-{end}")
            else:
                # pass
                bad_line += 1
                print(f"Annotation {tid} is incorrect: '{entity_text}' not found at indices {start}-{end}. Found '{substring}' instead.")

    if bad_line:
        print(txt_path)
    return bad_line


In [ ]:
# directory_path = '/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_swapped_sentences_extracted/bruk_changed'

# files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) 
#                   if os.path.isfile(os.path.join(directory_path, f)) and ".txt" in f]

In [ ]:
# for f in os.listdir(directory_path):
#     file_path = os.path.join(directory_path, f)
    
#     # Check if it's a file and contains the word 'filtered' in the name
#     if os.path.isfile(file_path) and "filtered" in f:
#         os.remove(file_path)  # Delete the file
#         print(f"Deleted: {file_path}")

In [ ]:
directory_path_bruk = '/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_swapped_sentences_extracted/bruk_changed'

filtered_files_bruk = [os.path.join(directory_path_bruk, f) for f in os.listdir(directory_path_bruk) 
                  if os.path.isfile(os.path.join(directory_path_bruk, f)) and "filtered.txt" in f]


In [ ]:
counter = 0

for f in filtered_files_bruk:
    bad_file = check_entity_indices(f, f.replace(".txt", ".ann"))
    if bad_file:
        counter += 1

In [ ]:
counter

In [ ]:
len("""Так вважає аналітикиня Любов Жмеренецька ( див . « Українську правду » ) .

« Найщиріші бажання поодиноких політичних груп та лідерів покращити ситуацію згасають , стикаючись із фундаментальними принципами старої системи , - підкреслює аналітикиня . - Фрагментарні реформи наштовхуються на саботаж громадян та управлінців , нівелюються шляхом використання розгалуженої системи законодавчих норм , що роками виписувалися для реалізації корупційних схем .

Політологиня з РФ Тетяна Чижова зазначає , що « в суспільстві зростає націоналізм , підживлений патріотичною риторикою .

Кажуть , для них іконою здатна стати Ольга Гіркіна ( колишня міністерка оборони самопроголошеної ДНР , російська офіцерка . - Ред . ) .

Під загрозою весь Північний Кавказ » , - підкреслює політологиня .

Насамперед від місцевих чиновниць і силовиць .

Радниця Президента України Ірина Медведєва ( посилаємося на ranok . ictv . ua ) нагадує : нещодавно експертки і політики запевняли , що """)

In [ ]:
len("""Камінь , розцяцькований темними крапками , """)

## ng

<!-- /Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner_orig/ng/6b71dda907fe.txt -->

In [ ]:
directory_path = '/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_swapped_sentences_extracted/ng_changed'

filtered_files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) 
                  if os.path.isfile(os.path.join(directory_path, f)) and "filtered.txt" in f]

In [ ]:
len(filtered_files)

In [1272]:
counter = 0
avg_bad_lines = 0

for f in filtered_files:
    bad_line = check_entity_indices(f, f.replace(".txt", ".ann"))
    if bad_line:
        avg_bad_lines += bad_line
        counter += 1

In [1246]:
avg_bad_lines/counter

49.0

In [1247]:
counter

1

In [ ]:
len("""За словами директорки «Національного туристичного офісу» Олександри Литвяк, німецька сторона запропонувала чотирьом містам України безкоштовну участь у експозиції, виділивши загалом 450 кв м площі.
В її підготовці візьмуть участь як державні """)

## T# Nornalization

delete all old orig files

In [ ]:
import os

def delete_ann_files(directory):
    for filename in os.listdir(directory):
        if filename.endswith("_1.ann"):
            file_path = os.path.join(directory, filename)
            os.remove(file_path)
            print(f"Deleted: {filename}")
        elif filename.endswith("_1.txt"):
            file_path = os.path.join(directory, filename)
            os.remove(file_path)
            print(f"Deleted: {filename}")


# directory = "/Users/linndfors/study/diploma/data_to_change_T_numbers_NER/bruk_changed"
directory = "/Users/linndfors/study/diploma/data_to_change_T_numbers_NER/ng_changed" # копія /Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_swapped_sentences_extracted

delete_ann_files(directory)

In [ ]:
def count_files(directory):
    return len([f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))])

directory = "/Users/linndfors/study/diploma/test_change_dir/ng_changed" # копія /Users/linndfors/study/diploma/data_to_change_T_numbers_NER/ng_changed
file_count = count_files(directory)
print(file_count)

548


renumbered

In [4]:
import os
import re

def renumber_ann_file(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()
    
    # Extract and sort T-annotations
    t_annotations = [line for line in lines if line.startswith("T")]
    others = [line for line in lines if not line.startswith("T")]
    
    # Renumber T-annotations sequentially
    new_t_annotations = []
    t_map = {}
    for i, line in enumerate(t_annotations, start=1):
        old_t_num = re.match(r"T\d+", line).group()
        new_t_num = f"T{i}"
        t_map[old_t_num] = new_t_num
        new_t_annotations.append(re.sub(r"T\d+", new_t_num, line, count=1))
    
    # Replace old T-numbers in other annotation references
    new_other_annotations = []
    for line in others:
        for old_t, new_t in t_map.items():
            line = line.replace(old_t, new_t)
        new_other_annotations.append(line)
    
    # Write back to file
    with open(file_path, "w", encoding="utf-8") as f:
        f.writelines(new_t_annotations + new_other_annotations)

def renumber_directory(directory):
    for filename in os.listdir(directory):
        if filename.endswith(".ann"):
            file_path = os.path.join(directory, filename)
            renumber_ann_file(file_path)
            print(f"Processed: {filename}")

In [ ]:
# directory = "/Users/linndfors/study/diploma/data_with_T_changed/bruk_changed"
# directory = "/Users/linndfors/study/diploma/data_with_T_changed/ng_changed"

# renumber_directory(directory)

Processed: 163e835b7aef_1_filtered.ann
Processed: 369c3876ae3d_1_filtered.ann
Processed: 535022e2bdd2_1_filtered.ann
Processed: 03b52358982b_1_filtered.ann
Processed: 00b1a798bebf_1_filtered.ann
Processed: 12ca40b4ecac_1_filtered.ann
Processed: 4ea65724096f_1_filtered.ann
Processed: 605667093f7b_1_filtered.ann
Processed: 1257e7421756_1_filtered.ann
Processed: e54ada2b9649_1_filtered.ann
Processed: fc5b5a35776f_1_filtered.ann
Processed: f62160cb3ef6_1_filtered.ann
Processed: ac7d3bf88e39_1_filtered.ann
Processed: e32c3effa79e_1_filtered.ann
Processed: 2980d5c34788_1_filtered.ann
Processed: e0c4e3168c81_1_filtered.ann
Processed: 01cfbeca9827_1_filtered.ann
Processed: 61267204736d_1_filtered.ann
Processed: 357039d66ab7_1_filtered.ann
Processed: 36aa8022709f_1_filtered.ann
Processed: a767be7c3792_1_filtered.ann
Processed: cffa0e3aa90f_1_filtered.ann
Processed: 003d28360166_1_filtered.ann
Processed: bf6104a5e8cc_1_filtered.ann
Processed: 0e6b8eb14491_1_filtered.ann
Processed: 8c6db2873e45_1

## Extract only JOB and PERS

In [1290]:
def filter_ann_file(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()
    
    # Keep only JOB and PERS entities
    filtered_lines = [line for line in lines if re.match(r"T\d+\t(JOB|PERS)\b", line)]
    
    # Save filtered content back to the original file
    with open(file_path, "w", encoding="utf-8") as f:
        f.writelines(filtered_lines)
    
    print(f"Filtered file updated: {file_path}")

def process_directory(directory):
    for filename in os.listdir(directory):
        if filename.endswith(".ann"):
            file_path = os.path.join(directory, filename)
            filter_ann_file(file_path)

# source_directory = "/Users/linndfors/study/diploma/test_change_dir/ng_changed"
source_directory = "/Users/linndfors/study/diploma/test_change_dir/bruk_changed"

process_directory(source_directory)

Filtered file updated: /Users/linndfors/study/diploma/test_change_dir/bruk_changed/4b1efc986bb8_1_filtered.ann
Filtered file updated: /Users/linndfors/study/diploma/test_change_dir/bruk_changed/5e10574a909c_1_filtered.ann
Filtered file updated: /Users/linndfors/study/diploma/test_change_dir/bruk_changed/18ec3aea7ed7_1_filtered.ann
Filtered file updated: /Users/linndfors/study/diploma/test_change_dir/bruk_changed/ef1b6bd52747_1_filtered.ann
Filtered file updated: /Users/linndfors/study/diploma/test_change_dir/bruk_changed/c6807dbff658_1_filtered.ann
Filtered file updated: /Users/linndfors/study/diploma/test_change_dir/bruk_changed/5710814a4a84_1_filtered.ann
Filtered file updated: /Users/linndfors/study/diploma/test_change_dir/bruk_changed/47eb526780b0_1_filtered.ann
Filtered file updated: /Users/linndfors/study/diploma/test_change_dir/bruk_changed/13e0cb95075a_1_filtered.ann
Filtered file updated: /Users/linndfors/study/diploma/test_change_dir/bruk_changed/38603af07530_1_filtered.ann
F

In [1291]:
renumber_directoy(source_directory)

Processed: 4b1efc986bb8_1_filtered.ann
Processed: 5e10574a909c_1_filtered.ann
Processed: 18ec3aea7ed7_1_filtered.ann
Processed: ef1b6bd52747_1_filtered.ann
Processed: c6807dbff658_1_filtered.ann
Processed: 5710814a4a84_1_filtered.ann
Processed: 47eb526780b0_1_filtered.ann
Processed: 13e0cb95075a_1_filtered.ann
Processed: 38603af07530_1_filtered.ann
Processed: 6b5d4229b548_1_filtered.ann
Processed: e0493ec8466b_1_filtered.ann
Processed: e023d6d6f1a3_1_filtered.ann
Processed: 164940c99b86_1_filtered.ann
Processed: 548f66f38090_1_filtered.ann
Processed: 755c890a06fb_1_filtered.ann
Processed: efc2665e4bed_1_filtered.ann
Processed: fabc18a7d8fd_1_filtered.ann
Processed: 326978d2c58c_1_filtered.ann
Processed: 9df8bcf8e17e_1_filtered.ann
Processed: f4ca58272dbf_1_filtered.ann
Processed: bc755153f4f7_1_filtered.ann
Processed: b987128d15d3_1_filtered.ann
Processed: d3669729136c_1_filtered.ann
Processed: b84c8a869b51_1_filtered.ann
Processed: e21ddb9efb07_1_filtered.ann
Processed: d530671576f8_1

renaming

In [1294]:
def rename_files(directory):
    for filename in os.listdir(directory):
        if "_1_filtered.ann" in filename:
            new_filename = filename.replace("_1_filtered.ann", "_1.ann")
            old_path = os.path.join(directory, filename)
            new_path = os.path.join(directory, new_filename)
            os.rename(old_path, new_path)
            print(f"Renamed: {filename} -> {new_filename}")
        elif "_1_filtered.txt" in filename:
            new_filename = filename.replace("_1_filtered.txt", "_1.txt")
            old_path = os.path.join(directory, filename)
            new_path = os.path.join(directory, new_filename)
            os.rename(old_path, new_path)
            print(f"Renamed: {filename} -> {new_filename}")


directory = "/Users/linndfors/study/diploma/test_change_dir/bruk_changed"
# directory = "/Users/linndfors/study/diploma/test_change_dir/ng_changed"


rename_files(directory)

Renamed: c28168a23912_1_filtered.txt -> c28168a23912_1.txt
Renamed: 4b1efc986bb8_1_filtered.ann -> 4b1efc986bb8_1.ann
Renamed: 5e10574a909c_1_filtered.ann -> 5e10574a909c_1.ann
Renamed: 1484f7813550_1_filtered.txt -> 1484f7813550_1.txt
Renamed: 232be2c6b5c2_1_filtered.txt -> 232be2c6b5c2_1.txt
Renamed: 92d38b895fa4_1_filtered.txt -> 92d38b895fa4_1.txt
Renamed: 17d3d678df81_1_filtered.txt -> 17d3d678df81_1.txt
Renamed: 18ec3aea7ed7_1_filtered.ann -> 18ec3aea7ed7_1.ann
Renamed: ef1b6bd52747_1_filtered.ann -> ef1b6bd52747_1.ann
Renamed: dc1202fd9850_1_filtered.txt -> dc1202fd9850_1.txt
Renamed: d530671576f8_1_filtered.txt -> d530671576f8_1.txt
Renamed: c6807dbff658_1_filtered.ann -> c6807dbff658_1.ann
Renamed: e21ddb9efb07_1_filtered.txt -> e21ddb9efb07_1.txt
Renamed: b84c8a869b51_1_filtered.txt -> b84c8a869b51_1.txt
Renamed: 5710814a4a84_1_filtered.ann -> 5710814a4a84_1.ann
Renamed: 47eb526780b0_1_filtered.ann -> 47eb526780b0_1.ann
Renamed: d3669729136c_1_filtered.txt -> d3669729136c_1.t

In [1296]:
file_count = count_files("/Users/linndfors/study/diploma/test_change_dir/ng_changed")
print(file_count)

548


In [1320]:
file_count = count_files(dir)
print(file_count)

596


In [1321]:
524+596

1120

In [1316]:
596 + 548 + 524 + 262

1930

Check

In [1322]:
from pathlib import Path

directory_path_bruk = Path("/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/bruk")

filenames_bruk = [file.name for file in directory_path_bruk.iterdir() if file.is_file()]

directory_path_ng = Path("/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/ng")

filenames_ng = [file.name for file in directory_path_bruk.iterdir() if file.is_file()]

In [1323]:
ng_and_bruk = filenames_bruk + filenames_ng

In [1325]:
len(ng_and_bruk)

1048